## Coleta de evevento

In [9]:
## Bibliotecas
import yfinance as yf
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import zipfile
import io
import pandas as pd
import numpy as np
from difflib import get_close_matches
import unidecode
import json


In [3]:
load_dotenv()
#userName = os.getenv("USERNAME")
#password = os.getenv("PASSWORD")
userName ="aluno.thiago.nunes" # os.getenv("USERNAME")
password = "NLPfinance2@23" #os.getenv("PASSWORD")

In [105]:
def EventsDate(ticker, userName="aluno.thiago.nunes", password="NLPfinance2%4023", startDate='01012010', endDate="01012024"):
  url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
  querystring = {"code":"import_data"}
  payload = f"username={userName}&password={password}&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D{startDate}%26data_fim%3D{endDate}%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D5%26f_v%3D1%26papel%3D{ticker}%26indic%3DNOME_EMPRESA%2BRL%2BLL%2BEBITDA%2BDATA_PUBLICACAO%2BPRECO_ABERTURA%2BPRECO_FECHAMENTO%26periodicidade%3Dtri%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dd%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
  headers = {'Content-Type': 'application/x-www-form-urlencoded'}
  response = requests.request("POST", url, data=payload, headers=headers, params=querystring)
  data = json.loads(response.text)
  df = pd.DataFrame(data["tables"]["tab0"]).T
  novas_colunas = ["Data", "Empresa", "Receita", "Lucro", "EBITDA", "Data_Publicacao", "Preco_Abertura", "Preco_fechamento", "Consolidado", "Convencao", "Moeda", "Data_Demonstracao", "Meses", "Data_Analise"]
  df.columns = novas_colunas
  df = df.drop("lin0")
  df['Data_Publicacao'] = pd.to_datetime(df['Data_Publicacao'], errors = 'coerce')
  df.reset_index(drop=True, inplace=True)
  df['Data_Publicacao'] = pd.to_datetime(df['Data_Publicacao'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
  return df

def EventsDate2(ticker, userName=userName, password=password, startDate='01012010', endDate="01012024"):
  url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
  querystring = {"code":"import_data"}
  payload = f"username={userName}&password={password}&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D{startDate}%26data_fim%3D{endDate}%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D5%26f_v%3D1%26papel%3D{ticker}%26indic%3DNOME_EMPRESA%2BRL%2BLL%2BEBITDA%2BDATA_PUBLICACAO%2BPRECO_ABERTURA%2BPRECO_FECHAMENTO%26periodicidade%3Dtri%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dd%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
  headers = {'Content-Type': 'application/x-www-form-urlencoded'}
  response = requests.request("POST", url, data=payload, headers=headers, params=querystring)
  data = json.loads(response.text)
  df = pd.DataFrame(data["tables"]["tab0"]).T
  novas_colunas = ["Data", "Empresa", "Receita", "Lucro", "EBITDA", "Data_Publicacao", "Preco_Abertura", "Preco_fechamento", "Consolidado", "Convencao", "Moeda", "Data_Demonstracao", "Meses", "Data_Analise"]
  df.columns = novas_colunas
  df = df.drop("lin0")
  df['Data_Publicacao'] = pd.to_datetime(df['Data_Publicacao'], errors = 'coerce', format='%d/%m/%Y')
  df.reset_index(drop=True, inplace=True)
  df['Data_Publicacao'] = pd.to_datetime(df['Data_Publicacao'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
  return df

def busca_date_pub(ano, codecvm):

    """
    Busca dados de publicação no site da CVM para o ano e código CVM especificados.

    Args:
        ano (int): O ano dos dados.
        codecvm (str): O código CVM da empresa.

    Returns:
        list: Lista de tuplas contendo os campos Assunto e Data_Entrega para o código CVM especificado.

    Example:
        >>> busca_date_pub(2023, '019348')
        [('Informações consolidadas dos exercícios', '2023-05-15'), ...]
    """
    url = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/IPE/DADOS/ipe_cia_aberta_%d.zip' % ano
    r = requests.get(url)
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    file = zf.namelist()
    zf = zf.open(file[0])
    lines = zf.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in lines]
    lines = [i.split(';') for i in lines]

    # Header mapping to find indexes of relevant columns
    header = lines[0]
    idx_codigo_cvm = header.index('Codigo_CVM')
    idx_assunto = header.index('Assunto')
    idx_data_entrega = header.index('Data_Entrega')

    # Strings de busca normalizadas para evitar problemas com acentos e maiúsculas/minúsculas
    busca1 = unidecode.unidecode("Informações consolidadas dos exercícios").lower()
    busca2 = unidecode.unidecode("Informações sobre o resultado").lower()
    busca3 = unidecode.unidecode("Relatório de Análise Gerencial").lower()

    result = []
    for line in lines[1:]:
        if line[idx_codigo_cvm] == codecvm:
            assunto_normalizado = unidecode.unidecode(line[idx_assunto]).lower()
            if busca1 in assunto_normalizado or busca2 in assunto_normalizado or busca3 in assunto_normalizado:
                result.append((line[idx_codigo_cvm], line[idx_assunto], line[idx_data_entrega]))

    # Convertendo o resultado em um DataFrame
    df_result = pd.DataFrame(result, columns=['Codigo_CVM', 'Assunto', 'Data_Entrega'])

    return df_result

In [115]:
ticker="PETR4.SA"
userName="aluno.thiago.nunes"
password="NLPfinance2%4023"
startDate="01012010"
endDate="01012024"

In [116]:
url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
querystring = {"code":"import_data"}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = f"username={userName}&password={password}&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D{startDate}%26data_fim%3D{endDate}%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D5%26f_v%3D1%26papel%3D{ticker}%26indic%3DNOME_EMPRESA%2BRL%2BLL%2BEBITDA%2BDATA_PUBLICACAO%2BPRECO_ABERTURA%2BPRECO_FECHAMENTO%26periodicidade%3Dtri%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dd%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
response = requests.request("POST", url, data=payload, headers=headers, params=querystring)
data = json.loads(response.text)

In [117]:
data

{'meta': {'date_time': '2024-08-29 21:27:32',
  'user': 'aluno.thiago.nunes',
  'server': 'minerva.comdinheiro.com.br',
  'feature': 'HistoricoIndicadoresFundamentalistas001',
  'execution_time': '0.472',
  'encode': 'utf8',
  'JSON_UNESCAPED_UNICODE': True,
  'JSON_PRETTY_PRINT': True,
  'JSON_UNESCAPED_SLASHES': True,
  'variables': {'get': {'ep': '1',
    'data_ini': '01012010',
    'data_fim': '01012024',
    'trailing': '12',
    'conv': 'MIXED',
    'moeda': 'MOEDA_ORIGINAL',
    'c_c': 'consolidado',
    'm_m': '1000000',
    'n_c': '5',
    'f_v': '1',
    'papel': 'PETR4.SA',
    'indic': 'NOME_EMPRESA RL LL EBITDA DATA_PUBLICACAO PRECO_ABERTURA PRECO_FECHAMENTO',
    'periodicidade': 'tri',
    'graf_tab': 'tabela',
    'desloc_data_analise': '1',
    'flag_transpor': '0',
    'c_d': 'd',
    'enviar_email': '0',
    'enviar_email_log': '0',
    'cabecalho_excel': 'modo1',
    'relat_alias_automatico': 'cmd_alias_01',
    'flag_export': 'json3',
    'ip_cliente': '189.62.46.1

In [112]:
df = pd.DataFrame(data["tables"]["tab0"]).T

In [113]:
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13
lin0,Data,BBDC4.SA\nNome da Empresa,BBDC4.SA\nReceita Líquida\n12 meses\nConsolida...,BBDC4.SA\nLucro Líquido\n12 meses\nConsolidado...,BBDC4.SA\nEBITDA\n12 meses\nConsolidado\n Milh...,BBDC4.SA\nDATA_PUBLICACAO,BBDC4.SA\nPRECO_ABERTURA,BBDC4.SA\nPRECO_FECHAMENTO,BBDC4.SA\nConsolidado\nou\nControlador,BBDC4.SA\nConvenção,BBDC4.SA\nMoeda,BBDC4.SA\nData da Demonstração,meses,Data da Análise
lin1,31/12/2023,,,,,31/12/9999,,,,,,31/12/2023,,02/01/2024
lin2,30/09/2023,,,,,31/12/9999,,,,,,30/09/2023,,02/10/2023
lin3,30/06/2023,,,,,31/12/9999,,,,,,30/06/2023,,03/07/2023
lin4,31/03/2023,,,,,31/12/9999,,,,,,31/03/2023,,03/04/2023
lin5,31/12/2022,,,,,31/12/9999,,,,,,31/12/2022,,02/01/2023
lin6,30/09/2022,,,,,31/12/9999,,,,,,30/09/2022,,03/10/2022
lin7,30/06/2022,,,,,31/12/9999,,,,,,30/06/2022,,01/07/2022
lin8,31/03/2022,,,,,31/12/9999,,,,,,31/03/2022,,01/04/2022
lin9,31/12/2021,,,,,31/12/9999,,,,,,31/12/2021,,03/01/2022


In [78]:
date_df = EventsDate(ticker='xxx.SA')

In [79]:
date_df

,Data,Empresa,Receita,Lucro,EBITDA,Data_Publicacao,Preco_Abertura,Preco_fechamento,Consolidado,Convencao,Moeda,Data_Demonstracao,Meses,Data_Analise
0,31/12/2023,,,,,NaN,,,,,,31/12/2023,,02/01/2024
1,30/09/2023,,,,,NaN,,,,,,30/09/2023,,02/10/2023
2,30/06/2023,,,,,NaN,,,,,,30/06/2023,,03/07/2023
3,31/03/2023,,,,,NaN,,,,,,31/03/2023,,03/04/2023
4,31/12/2022,,,,,NaN,,,,,,31/12/2022,,02/01/2023
5,30/09/2022,,,,,NaN,,,,,,30/09/2022,,03/10/2022
6,30/06/2022,,,,,NaN,,,,,,30/06/2022,,01/07/2022
7,31/03/2022,,,,,NaN,,,,,,31/03/2022,,01/04/2022
8,31/12/2021,,,,,NaN,,,,,,31/12/2021,,03/01/2022
9,30/09/2021,,,,,NaN,,,,,,30/09/2021,,01/10/2021


In [39]:
date_df
#df = pd.to_datetime(df['Data_Publicacao'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')['Data_Publicacao'] = pd.to_datetime(df['Data_Publicacao'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

df = pd.to_datetime(date_df['Data'], format= '%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [109]:
busca_date_pub(2020, 19348)